# 作業

實作各種不一樣的方式來調整飽和 / 對比 / 明亮

1. 改變 color space 來調整飽和度
2. 實作直方圖均衡
3. alpha/ beta 調整對比 / 明亮

In [ ]:
import cv2
import numpy as np

img_path = 'lena.png'

# 以彩色圖片的方式載入
image_gbr = cv2.imread('lena.png', cv2.IMREAD_COLOR)
image_hsv = cv2.cvtColor(image_gbr, cv2.COLOR_BGR2HSV)

## 改變飽和度

- 轉換成 HSV color space, 改變 s channel 的值

In [ ]:
change_percentage = 0.2

# 針對飽和度的值做改變，超過界線 0~1 的都會 bound
# 在 HLS color space 減少飽和度
image_hsv_down = image_hsv.astype('float32')

image_hsv_down[..., 1] = image_hsv_down[..., 1]/255 - change_percentage


height, width, channel = image_hsv.shape
image_hsv_down[... , 1] = (image_hsv_down[... , 1] *  255)
image_hsv_down_protect = image_hsv_down.copy()

for j in range(height):
    for i in range(width):
        if image_hsv_down_protect[j,i,1] < 0:
            image_hsv_down_protect[j,i,1] = 0


image_hsv_down[image_hsv_down[..., 1] < 0] = 0
image_hsv_down          = image_hsv_down.astype('uint8')
image_hsv_down_protect  = image_hsv_down_protect.astype('uint8')

image_hsv_down          = cv2.cvtColor(image_hsv_down, cv2.COLOR_HSV2BGR)
image_hsv_down_protect  = cv2.cvtColor(image_hsv_down_protect, cv2.COLOR_HSV2BGR)


image_hsv_up            = image_hsv.astype('float32')
image_hsv_up[..., 1]    = image_hsv_up[..., 1]/255 + change_percentage


height, width, channel  = image_hsv.shape
image_hsv_up[... , 1]   = (image_hsv_up[... , 1] *  255)

for j in range(height):
    for i in range(width):
        if image_hsv_up[j,i,1] > 255:
            image_hsv_up[j,i,1] = 255

image_hsv_up  = image_hsv_up.astype('uint8')
image_hsv_up  = cv2.cvtColor(image_hsv_up, cv2.COLOR_HSV2BGR)


img_hsv_change = np.hstack((image_gbr[:,:,:],image_hsv_down_protect[:, :, :],image_hsv_up[:, :, :]))
cv2.imshow('satuation reduced', img_hsv_change)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 直方圖均衡

1. case 1: 把彩圖拆開對每個 channel 個別做直方圖均衡再組合起來
2. case 2: 轉換 color space 到 HSV 之後對其中一個 channel 做直方圖均衡

In [ ]:
# 每個 channel 個別做直方圖均衡
image_g = image_gbr[:,:,0].copy()
image_b = image_gbr[:,:,1].copy()
image_r = image_gbr[:,:,2].copy()
# 灰階圖片直方圖均衡
img_g_equal = cv2.equalizeHist(image_g)
img_b_equal = cv2.equalizeHist(image_b)
img_r_equal = cv2.equalizeHist(image_r)

# 組合經過直方圖均衡的每個 channel
img_bgr_equal = np.array([img_g_equal, img_b_equal, img_r_equal])
img_bgr_equal = img_bgr_equal.transpose(1,2,0)

image_h = image_hsv[:,:,0].copy()
image_s = image_hsv[:,:,1].copy()
image_v = image_hsv[:,:,2].copy()

# 灰階圖片直方圖均衡
img_v_equal = cv2.equalizeHist(image_v)

img_hsv_equal = np.array([image_h, image_s, img_v_equal])
img_hsv_equal = img_hsv_equal.transpose(1,2,0)

img_hsv_equal = cv2.cvtColor(img_hsv_equal,cv2.COLOR_HSV2BGR)

img_bgr_equalHist = np.hstack((image_gbr, img_bgr_equal, img_hsv_equal))
cv2.imshow('bgr equal histogram', img_bgr_equalHist)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 調整對比 / 明亮

In [ ]:
# alpha: 控制對比度 (1.0~3.0)
# beta: 控制明亮度 (0~255)
add_contrast = cv2.convertScaleAbs(image_gbr, alpha=0.5, beta=0)
add_lighness = cv2.convertScaleAbs(image_gbr, alpha=1.0, beta=30)

# 組合圖片 + 顯示圖片
img_contrast_light = np.hstack((image_gbr, add_contrast, add_lighness))
cv2.imshow('adjust contrast and brighness', img_contrast_light)
cv2.waitKey(0)
cv2.destroyAllWindows()